In [58]:
import pandas as pd
import numpy as np
import pandas as pd
from pathlib import Path
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import PIL

In [59]:
root = Path('/hpc/group/kamaleswaranlab/EmoryDataset/Images/chest_xrays')

YEARS = [2015, 2016, 2017, 2018, 2019, 2020, 2021]
image_dir = "extracted-images"
metadata = "metadata_with_supertables_filtered_notes_filtered_matching_times_min_series_img_paths_datetime_hashed.csv"
notes = "all_notes.csv"

In [60]:
notesdf = pd.read_csv(root / notes)

/tmp/ipykernel_3140846/3106802520.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  notesdf = pd.read_csv(root / notes)


In [61]:
long_data = root / 'longitudinal_data_corrected'
embeddings_dir = long_data / 'image_embeddings'
ehr_dir = long_data / 'ehr_matrices'
notes_dir = long_data / 'notes'


In [62]:
encounters = list(ehr_dir.glob("*.npy"))
print(len(encounters))

17690


### Prepare Notes for input to chexpert

In [45]:
selected_accs = []
for i in range(len(encounters)):
    supertable_path = root / 'matched_supertables_with_images' / f"{str(encounters[i].stem).split('_')[0]}_image_interpolated.pickle"
    df = pd.read_pickle(supertable_path)
    acc_numbers = [x.split('_')[0] for x in df.loc[df['cxr_timing'].notna(), 'cxr_timing'].unique()]
    selected_accs.extend(acc_numbers)
    
np.save('selected_accs.npy', np.array(selected_accs))

In [48]:
notesdf_selected = notesdf.loc[notesdf.ACC_NBR.isin(selected_accs)]
notesdf_selected.to_csv(notes_dir / 'notes.csv', index = False)

In [47]:
len(selected_accs)

82449

In [49]:
notesdf_selected.shape

(86261, 12)

In [52]:
notesdf_selected['DOC_TEXT'].to_csv(notes_dir / 'chexpert_notes.csv')

In [55]:
notesdf_selected.loc[notesdf_selected['DOC_TEXT'].notna()]['DOC_TEXT'].to_csv(notes_dir / 'chexpert_notes_notna.csv')

In [56]:
num_chunks = 10
chunk_size = len(notesdf_selected) // num_chunks
remainder = len(notesdf_selected) % num_chunks

# Create chunks
chunks = []
start_idx = 0

for i in range(num_chunks):
    # Add one extra row to some chunks if the data doesn't divide evenly
    end_idx = start_idx + chunk_size + (1 if i < remainder else 0)
    chunk = notesdf_selected.iloc[start_idx:end_idx]
    chunk.loc[chunk['DOC_TEXT'].notna()]['DOC_TEXT'].to_csv(notes_dir / f'chexpert_notes_notna_{i}.csv')
    start_idx = end_idx

In [57]:
os.listdir(notes_dir)

['chexpert_notes_notna_1.csv',
 'chexpert_notes_notna.csv',
 'chexpert_notes_notna_9.csv',
 'chexpert_notes.csv',
 'chexpert_notes_notna_3.csv',
 'chexpert_notes_notna_6.csv',
 'chexpert_notes_notna_7.csv',
 'chexpert_notes_notna_2.csv',
 'chexpert_notes_notna_4.csv',
 'chexpert_notes_notna_0.csv',
 'chexpert_notes_notna_8.csv',
 'notes.csv',
 'chexpert_notes_notna_5.csv']

### Match chexpert labels 

In [63]:
notes = pd.read_csv(notes_dir / 'notes.csv')
chexpert_notes = pd.read_csv(notes_dir / 'chexpert_notes_notna.csv')


/tmp/ipykernel_3140846/3568618433.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  notes = pd.read_csv(notes_dir / 'notes.csv')


In [65]:
chexpert_notes.shape, notes.shape

((85677, 2), (86261, 12))

In [74]:
labelsdf = pd.DataFrame()

for i in range(10):
    chunk = pd.read_csv(notes_dir / f'emory_labels_{i}.csv')
    labelsdf = pd.concat((labelsdf, chunk))
    
labelsdf.shape

(85687, 15)

In [75]:
labelsdf = labelsdf.loc[labelsdf['Reports'] != 'DOC_TEXT']

In [77]:
chexpert_notes.iloc[0]['DOC_TEXT'] == labelsdf.iloc[0]['Reports']

True

In [80]:
notes = notes.loc[notes.DOC_TEXT.notna()]

In [84]:
notes_with_labels = notes.copy()

for column in labelsdf.columns:
    notes_with_labels[column] = labelsdf[column].values

In [85]:
notes_with_labels.shape

(85677, 27)

In [86]:
notes_with_labels.head()

,Unnamed: 0,ACC_NBR,PATIENT_ID,EMPI_NBR,ENCNTR_ID,HNAM_DOCUMENT_CLINICAL_ID,HNAM_DOCUMENT_CLINICAL_NM,DAY_VERIFIED,DOC_TEXT,DOC_ABSTRACT,...,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,60,2ed913e36727cd2fb59b7517d3b869b0228a615946c66e...,7f8076f1d64d54d3a2b08249526c1a0bd258264cccf274...,e0a020ae329369e5eeeb3edd22233e026112f53b4c237e...,883bf58480c3fb3c4555ddfa39a25acb31c9feee74bd9f...,32f51dc7f9e8d3674ca812cdb65c182d3265a827f88a80...,XR Chest 1 View Portable,04/27/2015,REPORT\r\rXR Chest 1 View Portable\n\nCLINICAL...,REPORT\r\rXR Chest 1 View Portable\n\nCLINICAL...,...,NaN,-1.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,1.0
1,63,7504d98282e19a4b7e7c5732523df7ede45c023125ffa4...,ec5356a6d22170a0da2be1932605833d72af5edf490a4a...,7edef2789d193cfeb73a668a8b59d87be7bc09b23d86bf...,e31bde44347cfb99afda94d362b2c7efc43bfa0d45fd6d...,32f51dc7f9e8d3674ca812cdb65c182d3265a827f88a80...,XR Chest 1 View Portable,04/27/2015,REPORT\r\rXR Chest 1 View Portable\n\nCLINICAL...,REPORT\r\rXR Chest 1 View Portable\n\nCLINICAL...,...,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,1.0
2,66,12d9bedb7e6fd0dcd41cd5bb76d76a287320cb17ebfbcd...,e424cb68d8f7d89095794f5b532ac0e9bc04f79bffcc2e...,0ebbec75d56effeb6f7f94f089e8e4cb500b7734489e03...,111f1384ccca3639056306f3145e58855d36ddb47ff698...,32f51dc7f9e8d3674ca812cdb65c182d3265a827f88a80...,XR Chest 1 View Portable,04/27/2015,REPORT\r\rXR Chest 1 View Portable\n\nCLINICAL...,REPORT\r\rXR Chest 1 View Portable\n\nCLINICAL...,...,1.0,NaN,NaN,-1.0,NaN,0.0,NaN,NaN,NaN,NaN
3,68,3ffae3bed0a25f6cba0a464e4dd7868ec4778e7dec4e6b...,3a4eb780db0ec2e991adc837320c5a9fcbdd4594845586...,c60a8d613ed473ebe37e3c3898d38abc8db41880cfd2f9...,bbdc13b41601ba68cd59fecb8afed3e138304f354f49cd...,32f51dc7f9e8d3674ca812cdb65c182d3265a827f88a80...,XR Chest 1 View Portable,04/27/2015,REPORT\r\rXR Chest 1 View Portable\n\nCLINICAL...,REPORT\r\rXR Chest 1 View Portable\n\nCLINICAL...,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0
4,73,c0fff3fc49fccd86b762d1657d7b95f9b69e556d8484ac...,22862ca8eed3091de4b45e5b1d54c41babefc3e3544c05...,a5bafb16270c0ff18a001ffa6e383e0b318107c30f495b...,2e726c5c77743ebf1c6d2b6c5902d2e698bc824fd70295...,32f51dc7f9e8d3674ca812cdb65c182d3265a827f88a80...,XR Chest 1 View Portable,04/27/2015,REPORT\r\rXR Chest 1 View Portable\n\nCLINICAL...,REPORT\r\rXR Chest 1 View Portable\n\nCLINICAL...,...,NaN,NaN,NaN,NaN,1.0,0.0,1.0,NaN,NaN,NaN


In [88]:
## Sanity Check

for idx, row in notes_with_labels.iterrows():
    if row['DOC_TEXT'] != row['Reports']:
        print(idx)

In [90]:
notes_with_labels.to_csv(notes_dir / 'notes_with_chexpert_labels.csv', index = False)